In [1]:
from databases import BDS, PosicaoDm1, Bawm,Crm,BaseExtrato,off_Reliance
from funcoes_datas import FuncoesDatas
from dateutil.relativedelta import relativedelta
import pandas as pd
from datetime import date, timedelta,datetime
from dateutil.relativedelta import relativedelta
import numpy as np
import math
from emailer import Email, EmailLer
from pretty_html_table import build_table
import funcoes

[base_dados.py directory]: C:\Temp\Foundation
[databases.py directory]: C:\Temp\Foundation


In [2]:
#Chama as classes do databases e as classes do funções data
rel_off = off_Reliance('Primeny')
crm = Crm()
bawm = Bawm()
datas = FuncoesDatas()
extrato = BaseExtrato()
bds = BDS()
fdt = FuncoesDatas()
#Puxando a ultima data do mes anterior
data = datetime.date(fdt.hoje() - relativedelta(days=fdt.hoje().day -1)+timedelta(days=-1))

In [3]:
#Retorna os produtos da base offshore Reliance
produtos_reliance=rel_off.produtos_isin()
produtos_reliance['ISIN']=produtos_reliance['ISIN'].str.lower()
produtos_reliance.columns=['ID_Produto','Ativo_Nome','id_bbg','ISIN','Classe','Subclasse','Categoria']
produtos_reliance['Ativo_Nome']=produtos_reliance['Ativo_Nome'].str.lower()
produtos_reliance = produtos_reliance.drop_duplicates(subset=['ID_Produto'])

#Retorna os produtos da base GPS para cruzar o ISIN e classificar os produtos ex-Reliance
produtos_gps = crm.produtos_off().rename(columns={'new_isin':'ISIN'})[['ISIN','new_indgroupaxysidname']]
produtos_gps['ISIN']=produtos_gps['ISIN'].str.lower()

#Verifica a classe que cada subclasse pertence (para ativos que estão na base GPS)
de_para_ativos = pd.read_excel('O:/SAO/CICH_All/Portfolio Management/20 - Offshore/De Para Ativos.xlsx',sheet_name='Tb De Para Subclass')

#Dicionario da classificação dos ativos dado pela area de investimentos produto a produto (produtos reliance) e unificando para a classificação gps.
dicio_classificacao_investimentos_rel = pd.read_excel('O:/SAO/CICH_All/Portfolio Management/20 - Offshore/De Para Ativos.xlsx',sheet_name='de_para_classificao')
dicio_classificacao_investimentos_rel=dicio_classificacao_investimentos_rel[['Classificação Reliance-Investimentos','Classificação GPS - v1']]
dicio_classificacao_investimentos_rel = dict(dicio_classificacao_investimentos_rel.values)

#Dicionario Reliance Classe + Subclasse + Categoria (feito por amostragem pela area de PM)
dicio_classe_sub_categoria = pd.read_excel('O:/SAO/CICH_All/Portfolio Management/20 - Offshore/De Para Ativos.xlsx',sheet_name='de_para_classificao')
dicio_classe_sub_categoria=dicio_classe_sub_categoria[['Classe_Subclasse_Categoria-Sistema','Classificação GPS - v2']]
dicio_classe_sub_categoria = dict(dicio_classe_sub_categoria.values)

#Saber a qual classe pertence cada subclasse
dicio_classe_subclasse_gps = pd.read_excel('O:/SAO/CICH_All/Portfolio Management/20 - Offshore/De Para Ativos.xlsx',sheet_name='de_para_classificao')
dicio_classe_subclasse_gps=dicio_classe_subclasse_gps[['Subclasse GPS','Classe']].astype('str')
dicio_classe_subclasse_gps['Subclasse GPS']=dicio_classe_subclasse_gps['Subclasse GPS'].apply(lambda x: x.replace(' -',''))
dicio_classe_subclasse_gps = dict(dicio_classe_subclasse_gps.values)

#Dicionario Reliance Classe + Subclasse + Categoria
dicio_projeto_estoque = pd.read_excel('O:/SAO/CICH_All/Portfolio Management/20 - Offshore/De Para Ativos.xlsx',sheet_name='de_para_classificao')
depara_rel=dicio_projeto_estoque[['Classe-Categoria','Proposta Base Única']]
depara_rel = dict(depara_rel.values)

depara_rel_gps=dicio_projeto_estoque[['Proposta Base Única','Classe Unificada']]
depara_rel_gps = dict(depara_rel_gps.values)

#Retorna os ativos vinculados aos fowards
fwd = rel_off.identificar_classe_fwd()

#dicio para classificar os fwd
dicio_fwd = pd.read_excel('O:/SAO/CICH_All/Portfolio Management/20 - Offshore/De Para Ativos.xlsx',sheet_name='de_para_classificao')
dicio_fwd=dicio_fwd[['Classe_Alocacao_sistema','Classe GPS']]
dicio_fwd = dict(dicio_fwd.values)


In [4]:
# #Mapeando os produtos que estão na posição reliance e segregando o que é posição e o que é estoque.

#Traz a ultima data do mes anterior

carteira_rel = rel_off.carteira_reliance(data)
carteira_rel['ISIN'] = carteira_rel['ISIN'].str.lower()    
carteira_rel['Nome_produto']=carteira_rel['Nome_produto'].str.lower()
carteira_rel=carteira_rel.rename(columns={'ISIN_x':'ISIN','asset_class':'subclasse','Nome_produto':'Ativo_Nome'})[['Ativo_Nome','cd_security']]
carteira_rel['Posicao'] = 'Possui Posição'
carteira_rel = pd.merge(left=produtos_reliance,right=carteira_rel,on='Ativo_Nome',how='left')
carteira_rel['Posicao'] = carteira_rel['Posicao'].fillna('Estoque')
carteira_rel['Categoria']=carteira_rel['Categoria'].fillna('')   
carteira_rel['Classe_Subclasse_Categoria']=carteira_rel['Classe']+'|'+carteira_rel['Subclasse']+'|'+carteira_rel['Categoria']
carteira_rel['Classe_Subclasse']=carteira_rel['Classe']+'|'+carteira_rel['Categoria']


In [5]:
carteira_rel.to_excel('carteira_rel.xlsx')

In [6]:
#Lê o excel que traz todas as classes e subclasses dos fundos (projeto base única de ativos)- Tratamento das informações dessa base que está em excel
classificacoes = pd.read_excel('O:/SAO/CICH_All/Portfolio Management/20 - Offshore/De Para Ativos.xlsx',sheet_name='TB Security')
classificacoes.columns =classificacoes.columns.str.strip()
classificacoes = classificacoes[['Ativo_Nome','refClassSubClass','ClassSubClass']].rename(columns={'refAtivo':'Ativo_Nome'})
classificacoes['Ativo_Nome']=classificacoes['Ativo_Nome'].str.lower().apply(lambda x: x.strip())
classificacoes= classificacoes[classificacoes['Ativo_Nome']!='']
classificacoes['refClassSubClass']=classificacoes['refClassSubClass'].astype('str')
classificacoes['refClassSubClass']=classificacoes['refClassSubClass'].apply(lambda x: x.strip())
classificacoes= classificacoes[classificacoes['refClassSubClass']!='']

In [7]:
#Consolida as classificações do projeto de base unica os produtos de toda a base.
carteira_rel = pd.merge(left=carteira_rel,right=classificacoes,on='Ativo_Nome',how='left').drop(columns={'cd_security'})
carteira_rel = pd.merge(left=carteira_rel, right=produtos_gps,on='ISIN',how='left').rename(columns={'new_indgroupaxysidname':'Subclasse_gps'})
carteira_rel['Subclasse_gps'] = carteira_rel['Subclasse_gps'].fillna('Não Classificado')
carteira_rel['Subclasse_gps']=carteira_rel['Subclasse_gps'].apply(lambda x: x.replace(' -',''))
carteira_rel['ClassSubClass'] = carteira_rel['ClassSubClass'].fillna('Não Classificado')
carteira_rel['Classe_gps'] = carteira_rel['Subclasse_gps'].apply(lambda x: dicio_classe_subclasse_gps[x] if x!='Não Classificado' else x)
carteira_rel['Classe_gps'] = carteira_rel.apply(lambda row: dicio_classificacao_investimentos_rel[row['ClassSubClass']] if (row['ClassSubClass']!='Não Classificado')&(row['Subclasse_gps'] =='Não Classificado') else row['Classe_gps'],axis=1)
carteira_rel['Classe']=carteira_rel['Classe_gps']
carteira_rel['Subclasse']=carteira_rel['Subclasse_gps']
carteira_rel = carteira_rel.drop_duplicates()

In [8]:
#Trazendo tudo o que é private equity e classificando produtos que possuem as palavras chave
lista_private_equity = ['investimento','comprometido','desembolsado','desembolsar','investido','investido','investment','commitment','called','pending']
classes=[]
for idx, row in carteira_rel.iterrows():
    if (any(map(row['Ativo_Nome'].__contains__,lista_private_equity))):
        classe = 'Iliq. P. Equity'
    else:
        classe = row['Classe']
    classes.append(classe)
carteira_rel['Classe']  =  classes
carteira_rel = carteira_rel.drop_duplicates()

#Verifica os ativos que são FWD e deixa igual do ativo que o mesmo oferece a profissão

carteira_rel = pd.merge(left = carteira_rel, right = fwd, left_on = 'ID_Produto',right_on = 'cd_security', how = 'left').drop(columns=['cd_security','FowardCdSecurityRefHedge'])
carteira_rel['asset_class'] = carteira_rel['asset_class'].astype('str')
carteira_rel['asset_class'] = carteira_rel['asset_class'].apply(lambda x : None if  x == 'nan' else dicio_fwd[x])
carteira_rel['Classe'].loc[carteira_rel['asset_class'].notnull()] = carteira_rel['asset_class'].loc[carteira_rel['asset_class'].notnull()]

In [9]:
#Separando produtos com posição
carteira_posicao = carteira_rel[carteira_rel['Posicao']=='Possui Posição'].reset_index(drop=True)
classe_subclasse_categoria = carteira_posicao['Classe_Subclasse_Categoria']
classe_categoria = carteira_posicao['Classe_Subclasse']


#Classificação feita por PM olhando 5 ativos de cada combinação (total de 200) e analisando qual classe mais de adqueda caso exista algum padrão
classificacao_pm=[]
for classe in classe_subclasse_categoria:
    try:
        classifica = dicio_classe_sub_categoria[classe]
    except:
        classifica = 'Não Classificado'
    classificacao_pm.append(classifica)    
        
carteira_posicao['Classe_gps-Classificacao_PM']=classificacao_pm

#Classificação feita no projeto de base de ativos com depara entre as classes/categoria.. posteriormente foi feito um depara para a classificaçã gps
classificacao_projeto_baseunica =[]
for classe in classe_categoria:
    try:
        classifica = depara_rel_gps[depara_rel[classe]]
    except:
        classifica = 'Não Classificado'
    classificacao_projeto_baseunica.append(classifica)    
        
carteira_posicao['Classe_gps-classificacao_projeto_baseunica']=classificacao_projeto_baseunica

#Se a classificação feita por PM e pela base unica forem a mesma consideramos essa, senão o ativo será marcado como não classificado.

carteira_posicao['Classe'] = carteira_posicao.apply(lambda row: row['Classe_gps-Classificacao_PM'] if (row['Classe_gps-Classificacao_PM']==row['Classe_gps-classificacao_projeto_baseunica'])&(row['Classe']=='Não Classificado') else row['Classe'],axis=1)

#Puxando a classificação manual que fizemos na BBG - Proximo Passo

# data_consulta = "{}.{}.{}".format(data.day,data.month,data.year)
# manual = pd.read_excel(f'O:/SAO/CICH_All/Portfolio Management/20 - Offshore/Classificacao/verificar_ativos_estoque-{data_consulta}.xlsx')[['ID_Produto','Classe']]
# carteira_posicao=pd.merge(left=carteira_posicao,right=manual,on='ID_Produto',how='left')
# # carteira_posicao['Classe']= carteira_posicao.apply(lambda row: row['Classe_y'] if row['Classe_x']=='Não Classificado' else row['Classe_x'],axis=1)
# # carteira_posicao = carteira_posicao.drop(columns={'Classe_y'})

#Analise do que conseguimos classificar
ativos_totais = len(carteira_posicao)
ativos_classificados = ativos_totais - len(carteira_posicao[carteira_posicao['Classe']=='Não Classificado'])
texto_1 = f'O total de ativos que possuímos posição é {ativos_totais}, sendo que {ativos_classificados} foram classficados - {str(round(ativos_classificados/ativos_totais,2)*100)} % \n Falta classificar {ativos_totais-ativos_classificados}'
print(texto_1)
carteira_posicao = carteira_posicao.drop_duplicates(subset =['Ativo_Nome'])

# #Planilha para classificar os ativos na bbg
# verificar= carteira_posicao[carteira_posicao['Classe']=='Não Classificado'].to_excel('O:/SAO/CICH_All/Portfolio Management/20 - Offshore/6-  Classificacao/verificar_ativos.xlsx')

#Planilha com a classificação da posicão
carteira_posicao.to_excel('O:/SAO/CICH_All/Portfolio Management/20 - Offshore/6-  Classificacao/classifica_posicao.xlsx')



O total de ativos que possuímos posição é 2330, sendo que 2190 foram classficados - 94.0 % 
 Falta classificar 140


KeyError: Index([''], dtype='object')

In [12]:
#Separando produtos que estão com estoque
carteira_estoque = carteira_rel[carteira_rel['Posicao']=='Estoque'].reset_index(drop=True)

classe_subclasse_categoria = carteira_estoque['Classe_Subclasse_Categoria']
classe_categoria = carteira_estoque['Classe_Subclasse']


#Classificação feita por PM olhando 5 ativos de cada combinação (total de 200) e analisando qual classe mais de adqueda caso exista algum padrão
classificacao_pm=[]
for classe in classe_subclasse_categoria:
    try:
        classifica = dicio_classe_sub_categoria[classe]
    except:
        classifica = 'Não Classificado'
    classificacao_pm.append(classifica)    
        
carteira_estoque['Classe_gps-Classificacao_PM']=classificacao_pm


#Classificação feita no projeto de base de ativos com depara entre as classes/categoria.. posteriormente foi feito um depara para a classificaçã gps
classificacao_projeto_baseunica =[]
for classe in classe_categoria:
    try:
        classifica = depara_rel_gps[depara_rel[classe]]
    except:
        classifica = 'Não Classificado'
    classificacao_projeto_baseunica.append(classifica)    
        
carteira_estoque['Classe_gps-classificacao_projeto_baseunica']=classificacao_projeto_baseunica

#Se a classificação feita por PM e pela base unica forem a mesma consideramos essa, senão o ativo será marcado como não classificado.

carteira_estoque['Classe'] = carteira_estoque.apply(lambda row: row['Classe_gps-Classificacao_PM'] if (row['Classe_gps-Classificacao_PM']==row['Classe_gps-classificacao_projeto_baseunica'])&(row['Classe']=='Não Classificado') else row['Classe_gps-classificacao_projeto_baseunica'] if (row['Classe_gps-Classificacao_PM']=='Não Classificado')&(row['Classe']=='Não Classificado') else row['Classe'],axis=1)

#Analise do que conseguimos classificar
ativos_totais = len(carteira_estoque)
ativos_classificados = ativos_totais - len(carteira_estoque[carteira_estoque['Classe']=='Não Classificado'])
texto_2 = f'O total de ativos que possuímos apenas em estoque é {ativos_totais}, sendo que {ativos_classificados} foram classficados - {str(round(ativos_classificados/ativos_totais,2)*100)} % \n Falta classificar {ativos_totais-ativos_classificados}'
print(texto_2)

#Planilha para classificar os ativos na bbg
verificar= carteira_estoque[carteira_estoque['Classe']=='Não Classificado'].to_excel('verificar_ativos_estoque.xlsx')

#Planilha com a classificação do estoque
carteira_estoque.to_excel('O:/SAO/CICH_All/Portfolio Management/20 - Offshore/6-  Classificacao/classifica_estoque.xlsx')

O total de ativos que possuímos apenas em estoque é 22088, sendo que 20381 foram classficados - 92.0 % 
 Falta classificar 1707


In [13]:
estoque = carteira_estoque[['ID_Produto','Ativo_Nome','id_bbg','ISIN','Classe','Posicao']].rename(columns={'Classe':'cd_asset_class_name'})
posicao = carteira_posicao[['ID_Produto','Ativo_Nome','id_bbg','ISIN','Classe','Posicao']].rename(columns={'Classe':'cd_asset_class_name'})
classificacao_total = pd.concat([posicao,estoque]).sort_values(by=['ID_Produto'])
classificacao_total = classificacao_total.drop_duplicates(subset=['ID_Produto'])
ativos_totais = len(classificacao_total)
ativos_classificados = ativos_totais - len(classificacao_total[classificacao_total['cd_asset_class_name']=='Não Classificado'])
texto_2 = f'O total de ativos que possuímos na base é {ativos_totais}, sendo que {ativos_classificados} foram classficados - {str(round(ativos_classificados/ativos_totais,2)*100)} % \n Falta classificar {ativos_totais-ativos_classificados}'
print(texto_2)

O total de ativos que possuímos na base é 24372, sendo que 22525 foram classficados - 92.0 % 
 Falta classificar 1847


In [14]:
contagem = classificacao_total.groupby(['Ativo_Nome','id_bbg', 'ISIN','cd_asset_class_name']).size().reset_index(name='counts')
contagem = contagem.sort_values(['cd_asset_class_name','counts'],ascending=False).groupby(['cd_asset_class_name']).head(10).dropna(subset=['cd_asset_class_name']).reset_index(drop=True)
lista = contagem['cd_asset_class_name'].drop_duplicates().to_list()
contagem.to_excel('amostragem_classificação.xlsx')

In [15]:
classificacao_total.to_excel('classificacao_total.xlsx')